In [1]:
import pandas as pd
import numpy as np
from epiweeks import Week
from numpy import log, exp
from scipy.stats import lognorm
from scipy.optimize import minimize
from matplotlib import use, get_backend
import aux_functions

#use('TkAgg', force=True)
from matplotlib import pyplot as plt
#print("Switched to:", get_backend())

Load preds: 

In [2]:
df_preds= pd.read_csv('results/forecast_quantiles_cases_v3.csv')

#df_preds = df_preds.loc[df_preds.model_id != 141]
df_preds.forecast_date = pd.to_datetime(df_preds.forecast_date)
df_preds.target_end_date = pd.to_datetime(df_preds.target_end_date)


#df_preds = df_preds.loc[df_preds.date <= Week(2025, 40).startdate().strftime('%Y-%m-%d')]
            
df_preds.head()

,model,forecast_date,target_end_date,horizon,lower_95,lower_80,lower_50,pred,upper_50,upper_80,upper_95,value
0,BPagano-RtDriven,2020-10-18,2020-10-24,1,252157.46981,309027.43446,360710.91108,418324.95041,475938.98974,527622.46637,584492.43102,485474.0
1,BPagano-RtDriven,2020-10-18,2020-10-31,2,250557.34760,325076.22060,392799.04150,468292.90380,543786.76609,611509.58699,686028.46000,572162.0
2,BPagano-RtDriven,2020-10-18,2020-11-07,3,236909.92449,332846.68480,420034.10156,517226.07830,614418.05504,701605.47179,797542.23210,777428.0
3,BPagano-RtDriven,2020-10-18,2020-11-14,4,213876.19851,330803.52595,437067.18958,555524.36963,673981.54967,780245.21330,897172.54074,1028914.0
4,BPagano-RtDriven,2020-10-25,2020-10-31,1,299044.13571,364922.81525,424793.42033,491534.04269,558274.66504,618145.27012,684023.94966,572162.0


In [3]:
len(df_preds)

6867

In [4]:
df_preds = df_preds.dropna()
df_preds = df_preds.reset_index(drop=True)

In [5]:
len(df_preds)

6791

In [6]:
6867-6791

76

### Estimate from the KL: 

In [7]:
l_pos = tuple([col for col in df_preds.columns if 'lower' in col])
u_pos = tuple([col for col in df_preds.columns if 'upper' in col])
the_pos = list(l_pos + u_pos)
l_pos = list(l_pos)
u_pos = list(u_pos)

lvls = (0.5, 0.8, 0.95)
p = [aux_functions.quantile_pair(lvl) for lvl in lvls]
p = [i for quantile in p for i in quantile]
p.append(0.5)
ps = p.copy()
ps.sort()


In [8]:
ps

[0.025000000000000022, 0.09999999999999998, 0.25, 0.5, 0.75, 0.9, 0.975]

In [9]:
k = 0
lowers = list(df_preds.loc[k, l_pos])
lowers

[np.float64(252157.46981), np.float64(309027.43446), np.float64(360710.91108)]

In [10]:
k = 0
lowers = list(df_preds.loc[k, l_pos])
J = len(lowers)
names_l = [name.split('_') for name in df_preds.loc[k, l_pos].index]
extracted_lvls = [float(parts[1]) for parts in names_l if len(parts) > 1]

uppers = list(df_preds.loc[k, u_pos])

intervals = pd.DataFrame({
    'level': [i / 100 for i in extracted_lvls],
    'lwr': lowers,
    'med': [df_preds.loc[k, 'pred']] * J,
    'upr': uppers
})

approxs = [[], []]
for j in range(0, J):
    aux = aux_functions.get_lognormal_pars2(med = intervals['med'][j],
                                           lwr = intervals['lwr'][j],
                                           upr = intervals['upr'][j],
                                           conf_level = intervals['level'][j],
                                           fn_loss = 'lower')
    approxs[0].append(aux[0])
    approxs[1].append(aux[1])

ln_approx = pd.DataFrame({
    'level': list(intervals['level']),
    'mu': approxs[0],
    'sigma': approxs[1]
})

ln_approx.head()

,level,mu,sigma
0,0.95,12.755427,0.162053
1,0.80,12.908661,0.208712
2,0.50,13.037166,0.357802


In [11]:
Opt_direct_KL = aux_functions.minimize_opt_fn1(method = 'L-BFGS-B',
                                               x0 = np.array([0.0, 0.0]),
                                               J = J,
                                               ln_approx = ln_approx)

direct_KL_pars = [Opt_direct_KL.x[0], exp(Opt_direct_KL.x[1])]


direct_KL_pars_closedForm = [sum(ln_approx['mu']/ln_approx['sigma']**2)/sum(1/ln_approx['sigma']**2), np.sqrt(J/sum(1/ln_approx['sigma']**2))]


In [12]:
direct_KL_pars

[np.float64(12.838487053878513), np.float64(0.2087465383230363)]

In [13]:
direct_KL_pars_closedForm

[12.83848705885016, np.float64(0.20874653350787842)]

In [14]:
ln_approx

,level,mu,sigma
0,0.95,12.755427,0.162053
1,0.80,12.908661,0.208712
2,0.50,13.037166,0.357802


In [15]:
df_preds.loc[0]

model                 BPagano-RtDriven
forecast_date      2020-10-18 00:00:00
target_end_date    2020-10-24 00:00:00
horizon                              1
lower_95                  252157.46981
lower_80                  309027.43446
lower_50                  360710.91108
pred                      418324.95041
upper_50                  475938.98974
upper_80                  527622.46637
upper_95                  584492.43102
value                         485474.0
Name: 0, dtype: object

In [16]:
lognorm.ppf(ps, scale = exp(direct_KL_pars_closedForm[0]),  s = direct_KL_pars_closedForm[1])

array([250033.79589231, 288072.54277363, 326991.56734468, 376429.8768655 ,
       433342.83311229, 491888.08774575, 566721.19739368])

Condições fixas para todo o método: 

out['est'] = lognorm.ppf(ps,
                              scale = exp(cdf_fit[0]),  # scale = exp(meanlog)
                              s = cdf_fit[1])

out.head(10)

In [17]:
def process_row(row):
    # pegar valores da linha
    lowers = list(row[l_pos])
    J = len(lowers)
    names_l = [name.split('_') for name in row[l_pos].index]
    extracted_lvls = [float(parts[1]) for parts in names_l if len(parts) > 1]
    
    uppers = list(row[u_pos])
    
    intervals = pd.DataFrame({
        'level': [i / 100 for i in extracted_lvls],
        'lwr': lowers,
        'med': [row['pred']] * J,
        'upr': uppers
    })
    
    approxs = [[], []]
    for j in range(0, J):
        aux = aux_functions.get_lognormal_pars2(med = intervals['med'][j],
                                               lwr = intervals['lwr'][j],
                                               upr = intervals['upr'][j],
                                               conf_level = intervals['level'][j],
                                               fn_loss = 'lower')
        approxs[0].append(aux[0])
        approxs[1].append(aux[1])
    
    ln_approx = pd.DataFrame({
        'level': list(intervals['level']),
        'mu': approxs[0],
        'sigma': approxs[1]
    })

    #Opt_direct_KL = aux_functions.minimize_opt_fn1(method = 'L-BFGS-B',
    #                                           x0 = np.array([0.0, 0.0]),
    #                                           J = J,
    #                                           ln_approx = ln_approx)

    #direct_KL_pars = [Opt_direct_KL.x[0], exp(Opt_direct_KL.x[1])]

    direct_KL_pars_closedForm = [sum(ln_approx['mu']/ln_approx['sigma']**2)/sum(1/ln_approx['sigma']**2), np.sqrt(J/sum(1/ln_approx['sigma']**2))]
    
    return pd.Series({
    'mu': direct_KL_pars_closedForm[0],
    'sigma': direct_KL_pars_closedForm[1]
})

In [18]:
%%time 

df_preds[['mu', 'sigma']] = df_preds.apply(process_row, axis=1)

df_preds.head()

C:\Users\alp19\Documentos\EMAp-Doutorado\Dropbox\Mosqlimate_COVID_HUB\harmonization_strategies_py_COVID\aux_functions.py:74: RuntimeWarning: divide by zero encountered in scalar divide
  attained_loss = abs(upr - tent_qs[1]) / upr
C:\Users\alp19\anaconda3\envs\D_M2\Lib\site-packages\scipy\optimize\_optimize.py:869: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):
C:\Users\alp19\Documentos\EMAp-Doutorado\Dropbox\Mosqlimate_COVID_HUB\harmonization_strategies_py_COVID\aux_functions.py:74: RuntimeWarning: divide by zero encountered in scalar divide
  attained_loss = abs(upr - tent_qs[1]) / upr
C:\Users\alp19\anaconda3\envs\D_M2\Lib\site-packages\scipy\optimize\_optimize.py:869: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):
C:\Users\alp19\Documentos\EMAp-Doutorado\Dropbox\Mosqlimate_COVID_HUB\harmonization_strategies_py_COVID\aux_functions.py:74: RuntimeWarning: divide by zero encoun

CPU times: total: 18min 33s
Wall time: 21min 11s


,model,forecast_date,target_end_date,horizon,lower_95,lower_80,lower_50,pred,upper_50,upper_80,upper_95,value,mu,sigma
0,BPagano-RtDriven,2020-10-18,2020-10-24,1,252157.46981,309027.43446,360710.91108,418324.95041,475938.98974,527622.46637,584492.43102,485474.0,12.838487,0.208747
1,BPagano-RtDriven,2020-10-18,2020-10-31,2,250557.34760,325076.22060,392799.04150,468292.90380,543786.76609,611509.58699,686028.46000,572162.0,12.925423,0.250265
2,BPagano-RtDriven,2020-10-18,2020-11-07,3,236909.92449,332846.68480,420034.10156,517226.07830,614418.05504,701605.47179,797542.23210,777428.0,12.991771,0.300785
3,BPagano-RtDriven,2020-10-18,2020-11-14,4,213876.19851,330803.52595,437067.18958,555524.36963,673981.54967,780245.21330,897172.54074,1028914.0,13.027909,0.352790
4,BPagano-RtDriven,2020-10-25,2020-10-31,1,299044.13571,364922.81525,424793.42033,491534.04269,558274.66504,618145.27012,684023.94966,572162.0,13.001782,0.205413


In [19]:
df_preds.isnull().sum()

model              0
forecast_date      0
target_end_date    0
horizon            0
lower_95           0
lower_80           0
lower_50           0
pred               0
upper_50           0
upper_80           0
upper_95           0
value              0
mu                 0
sigma              0
dtype: int64

In [20]:
df_preds[df_preds.isnull().any(axis=1)]

,model,forecast_date,target_end_date,horizon,lower_95,lower_80,lower_50,pred,upper_50,upper_80,upper_95,value,mu,sigma


In [21]:
col = 'sigma'
pos_inf_count = np.isposinf(df_preds[col]).sum()

# count only negative inf
neg_inf_count = np.isneginf(df_preds[col]).sum()

# count both +inf and -inf
total_inf_count = np.isinf(df_preds[col]).sum()

print("positive inf:", pos_inf_count)
print("negative inf:", neg_inf_count)
print("total inf:", total_inf_count)

positive inf: 0
negative inf: 0
total inf: 0


In [23]:
df_preds.to_csv('results/preds_KL_cases_v3.csv', index = False)